In [28]:
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 경로
path = '../data_raw/'

# 카드매출 데이터 불러오기
sales_2023_sexage = pd.read_csv(path + 'card_sales_2023_sexage.csv', encoding='cp949')
sales_2024_sexage = pd.read_csv(path + 'card_sales_2024_sexage.csv', encoding='cp949')
sales_2023_time = pd.read_csv(path + 'card_sales_2023_time.csv', encoding='cp949')
sales_2024_time = pd.read_csv(path + 'card_sales_2024_time.csv', encoding='cp949')
sales_2023_month = pd.read_csv(path + 'card_sales_2023_month.csv', encoding='cp949')
sales_2024_month = pd.read_csv(path + 'card_sales_2024_month.csv', encoding='cp949')

In [43]:
# 전처리 함수: 파일 유형별 처리 ('sexage', 'time', 'month')
def preprocess_sales_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:
    df = df.copy()

    # 공통 컬럼명 통일
    rename_map = {
        '매출금액합계': '매출금액',
        '매출합계(원)': '매출금액',
        '매출건수(건)': '매출건수',
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # sexage 데이터: 연령대/성별 처리
    if dataset_type == 'sexage':
        if '연령대' in df.columns:
            df['청년여부'] = df['연령대'].apply(lambda x: '청년' if x in ['20대', '30대'] else '기타')
        if '성별' in df.columns:
            df['성별'] = df['성별'].str.strip()

    # time 데이터: 시간대 → 야간 여부 처리
    elif dataset_type == 'time':
        if '시간대' in df.columns:
            df['야간여부'] = df['시간대'].apply(lambda x: '야간' if (x >= 18 or x < 6) else '주간')

    # month 데이터: 유입지 → 외지 여부 처리
    elif dataset_type == 'month':
        if '유입지' in df.columns:
            df['외지여부'] = df['유입지'].apply(lambda x: '외지' if x != '경남' else '내지')

    return df


In [44]:
# sexage
sexage_2023 = preprocess_sales_data(sales_2023_sexage, 'sexage')
sexage_2024 = preprocess_sales_data(sales_2024_sexage, 'sexage')

# time
time_2023 = preprocess_sales_data(sales_2023_time, 'time')
time_2024 = preprocess_sales_data(sales_2024_time, 'time')

# month
month_2023 = preprocess_sales_data(sales_2023_month, 'month')
month_2024 = preprocess_sales_data(sales_2024_month, 'month')


In [45]:
# 🔹 지표 ①: 청년소비비중
def calc_yci_1_청년소비비중(df):
    # 전체 매출 합계
    total_sales = df.groupby('시군구명')['매출금액'].sum()
    
    # 청년 매출 합계
    youth_sales = df[df['청년여부'] == '청년'].groupby('시군구명')['매출금액'].sum()
    
    # 비중 계산
    share = (youth_sales / total_sales).fillna(0)

    # 정규화
    norm = (share - share.min()) / (share.max() - share.min() + 1e-9)

    return pd.DataFrame({
        '청년소비비중': share,
        '청년소비비중_norm': norm
    })


In [46]:
yci1_2023 = calc_yci_1_청년소비비중(sexage_2023)
yci1_2023.head()


,청년소비비중,청년소비비중_norm
시군구명,,
거제시,0.252248,0.391685
거창군,0.238184,0.342199
고성군,0.208021,0.236075
김해시,0.295953,0.545455
남해군,0.281668,0.495194


In [47]:
YOUTH_CORE_CATEGORIES = ['식생활', '유통', '의료/미용', '자동차']


In [48]:
# 🔹 지표 ②: 청년핵심업종비중
def calc_yci_2_청년핵심업종비중(df):
    core_categories = ['식생활', '유통', '의료/미용', '자동차']
    
    # 청년 소비만 필터
    youth = df[df['청년여부'] == '청년']

    # 전체 청년 소비
    total_youth_sales = youth.groupby('시군구명')['매출금액'].sum()

    # 청년의 핵심업종 소비
    core_youth_sales = youth[youth['분류'].isin(core_categories)] \
                            .groupby('시군구명')['매출금액'].sum()

    # 비중 계산
    share = (core_youth_sales / total_youth_sales).fillna(0)

    # 정규화
    norm = (share - share.min()) / (share.max() - share.min() + 1e-9)

    return pd.DataFrame({
        '청년핵심업종비중': share,
        '청년핵심업종비중_norm': norm
    })


In [49]:
yci2_2023 = calc_yci_2_청년핵심업종비중(sexage_2023)
yci2_2023.head()

,청년핵심업종비중,청년핵심업종비중_norm
시군구명,,
거제시,0.883717,0.312288
거창군,0.931554,0.625929
고성군,0.899809,0.417795
김해시,0.922728,0.568064
남해군,0.883899,0.313485


In [50]:
import numpy as np

# 🔹 지표 ③: 청년소비다양성
def calc_yci_3_청년소비다양성(df):
    youth = df[df['청년여부'] == '청년']
    
    # 시군구별 업종 비율 계산
    grouped = youth.groupby(['시군구명', '분류'])['매출금액'].sum().reset_index()
    
    # 전체 매출 대비 업종 비율
    total_per_region = grouped.groupby('시군구명')['매출금액'].sum().reset_index()
    grouped = grouped.merge(total_per_region, on='시군구명', suffixes=('', '_total'))
    grouped['비율'] = grouped['매출금액'] / grouped['매출금액_total']
    
    # Shannon entropy 계산
    entropy = grouped.groupby('시군구명')['비율'].apply(
        lambda x: -np.sum(x * np.log2(x + 1e-9))
    )

    # 정규화
    norm = (entropy - entropy.min()) / (entropy.max() - entropy


SyntaxError: incomplete input (3849206786.py, line 21)

In [37]:
def calc_yci_5_성별균형성(df):
    df_youth = df[df['연령대'].isin(YOUTH_AGES)]
    grouped = df_youth.groupby(['시군구명', '성별'])['매출금액'].sum().reset_index()
    pivot = grouped.pivot(index='시군구명', columns='성별', values='매출금액').fillna(0)

    pivot['총합'] = pivot.sum(axis=1)
    pivot['남성비율'] = pivot['남성'] / pivot['총합']
    pivot['여성비율'] = pivot['여성'] / pivot['총합']
    pivot['성별균형성'] = 1 - abs(pivot['남성비율'] - pivot['여성비율'])
    return pivot[['성별균형성']]


In [38]:
def normalize_series(s):
    return (s - s.min()) / (s.max() - s.min() + 1e-9)  # 0으로 나눔 방지


In [41]:
def calculate_yci_score(df_sexage, df_month):
    # 전처리
    df_sexage = preprocess_sexage(df_sexage)

    # 개별 지표 계산
    df1 = calc_yci_1_청년소비비중(df_sexage)
    df2 = calc_yci_2_청년핵심업종비중(df_sexage)
    df3 = calc_yci_3_청년소비다양성(df_sexage)
    df4 = calc_yci_4_청년외지소비(df_month)
    df5 = calc_yci_5_성별균형성(df_sexage)

    # 병합
    df_all = df1.join(df2).join(df3).join(df4).join(df5)

    # 정규화
    for col in df_all.columns:
        df_all[col + '_norm'] = normalize_series(df_all[col])

    # 가중치 (정책적 중요도 기준)
    weights = {
        '청년소비비중_norm': 0.30,
        '청년핵심업종비중_norm': 0.25,
        '청년소비다양성_norm': 0.15,
        '청년외지소비비중_norm': 0.15,
        '성별균형성_norm': 0.15
    }

    # 가중합
    df_all['YCI_score'] = sum(df_all[col] * w for col, w in weights.items())
    df_all['YCI_score'] = df_all['YCI_score'] * 100  # 0~100 스케일

    return df_all[['YCI_score'] + list(weights.keys())]


In [42]:
# 예시: 2023년 기준 YCI 계산
yci_2023 = calculate_yci_score(sales_2023_sexage, sales_2023_month)
yci_2023.head()


/tmp/ipykernel_8396/2929928930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youth['핵심업종'] = df_youth['분류'].apply(lambda x: x if x in 청년_핵심_업종 else '기타')
/tmp/ipykernel_8396/3300425101.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  entropy = grouped.groupby('시군구명').apply(shannon_entropy).reset_index()


KeyError: '연령대'